# Use the Geocoder package

In [6]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

# getting data from internet
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text

# using beautiful soup to parse the HTML/XML codes.
soup = BeautifulSoup(raw_wikipedia_page,'xml')

In [7]:
# extracting the raw table inside that webpage
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

# print(table)

# extracting a clean form of the table
for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

In [8]:
unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 77


In [9]:
!pip install geocoder
import geocoder
print('geocoder has been successfully imported.')

Requirement not upgraded as not directly required: geocoder in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: click in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: ratelim in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: decorator in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ratelim->geocoder)
Requirement not upgraded as not directly required: chardet<3.1.0,

In [29]:
"Note: Geocoder not working so using csv"

'Note: Geocoder not working so using csv'

In [30]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_part2_temp.csv')
df_toronto.head(14)

,Borough,Neighbourhood,Postcode
0,North York,Hillcrest Village,M2H
1,Scarborough,"Cliffcrest, Cliffside",M1M
2,Etobicoke,"Mimico NW, The Queensway West",M8Z
3,York,Weston,M9N
4,North York,"Bathurst Manor, Wilson Heights",M3H
5,Downtown Toronto,"First Canadian Place, Underground city",M5X
6,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",M1P
7,Scarborough,"Morningside, West Hill",M1E
8,Etobicoke,Kingsview Village,M9R
9,West Toronto,Dovercourt Village,M6H


In [31]:
geo_spatial_data = pd.read_csv('https://cocl.us/Geospatial_data')
geo_spatial_data.rename(columns={'Postal Code':'Postcode'}, inplace=True)
geo_spatial_data.head(10)

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [32]:
df_toronto_new = pd.merge(df_toronto, geo_spatial_data, on='Postcode')
df_toronto_new.head(10)

,Borough,Neighbourhood,Postcode,Latitude,Longitude
0,North York,Hillcrest Village,M2H,43.803762,-79.363452
1,Scarborough,"Cliffcrest, Cliffside",M1M,43.716316,-79.239476
2,Etobicoke,"Mimico NW, The Queensway West",M8Z,43.628841,-79.520999
3,York,Weston,M9N,43.706876,-79.518188
4,North York,"Bathurst Manor, Wilson Heights",M3H,43.754328,-79.442259
5,Downtown Toronto,"First Canadian Place, Underground city",M5X,43.648429,-79.382280
6,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",M1P,43.757410,-79.273304
7,Scarborough,"Morningside, West Hill",M1E,43.763573,-79.188711
8,Etobicoke,Kingsview Village,M9R,43.688905,-79.554724
9,West Toronto,Dovercourt Village,M6H,43.669005,-79.442259


In [34]:
df_toronto_new.to_csv('toronto_base.csv')
df_toronto_new.head(10)

,Borough,Neighbourhood,Postcode,Latitude,Longitude
0,North York,Hillcrest Village,M2H,43.803762,-79.363452
1,Scarborough,"Cliffcrest, Cliffside",M1M,43.716316,-79.239476
2,Etobicoke,"Mimico NW, The Queensway West",M8Z,43.628841,-79.520999
3,York,Weston,M9N,43.706876,-79.518188
4,North York,"Bathurst Manor, Wilson Heights",M3H,43.754328,-79.442259
5,Downtown Toronto,"First Canadian Place, Underground city",M5X,43.648429,-79.382280
6,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",M1P,43.757410,-79.273304
7,Scarborough,"Morningside, West Hill",M1E,43.763573,-79.188711
8,Etobicoke,Kingsview Village,M9R,43.688905,-79.554724
9,West Toronto,Dovercourt Village,M6H,43.669005,-79.442259
